In [1]:
from game import Game2048
from game_env import Game2048Env

# Search content

In [2]:
# Definir modelo de NN por reforço
    # Métodos
        # Q-Learning
        # Monte Carlo
        # Gradiente de políticas
        # Aprendizado por diferença temporal
        # Trust Region Policy Optimization (TRPO)
# Definir parâmetros
    # Grid do jogo
# Definir penalidades
    # Tentar mover para uma direção sem movimentos válidos
    # Valores iguais/próximos muito dispersos? (viés?)
        # Aplicar penalidade apenas quando saiu do range (até 2 de distância?)
        # Verificar valores disponíveis no grid e contabilizar como valor próximo se estiver a até (3?) valores de distância
        # Não penalizar se valor for 2 ou 4.
# Definir recompensas
    # Juntar tiles
    # Manter valores da sequência em fileira ou próximos? (viés?)
    # 
# Definir estratégia de treino
    # Utilizar jogo já iniciado com mais tiles no grid? (aumentar aprendizado inicial sobre a regra do jogo)
        # Treinar o mesmo jogo com rotações diferentes

# Material:
# https://towardsdatascience.com/deep-q-learning-tutorial-mindqn-2a4c855abffc
# https://www.analyticsvidhya.com/blog/2019/04/introduction-deep-q-learning-python
# https://www.analyticsvidhya.com/blog/2019/01/monte-carlo-tree-search-introduction-algorithm-deepmind-alphago
# https://medium.com/@samina.amin/deep-q-learning-dqn-71c109586bae
# https://www.geeksforgeeks.org/implementing-deep-q-learning-using-tensorflow/

# Model

In [2]:
# pip install tensorflow==2.9.0 keras-rl2
# pip uninstall numpy -y
# pip install numpy==1.24.3

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Activation
from tensorflow.keras.optimizers import Adam

from rl.agents import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory

2025-03-09 13:52:58.100856: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-03-09 13:52:58.100893: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [7]:

nActions = 4
tableSize = 4
hiddenFeatures = 16
temporalWindow = 5
env = Game2048Env()

model = Sequential()
model.add(Flatten(input_shape=(temporalWindow, tableSize, tableSize)))

model.add(Dense(hiddenFeatures))
model.add(Activation(activation="relu"))
# model.add(Activation(activation="leaky_relu")) # Tune alpha
# model.add(Activation(activation="elu")) # Tune alpha

model.add(Dense(nActions))
model.add(Activation(activation="linear"))

policy = EpsGreedyQPolicy()
replayMemory = SequentialMemory(limit=100, window_length=temporalWindow)
dqn = DQNAgent(
    model=model,
    nb_actions=nActions,
    memory=replayMemory,
    nb_steps_warmup=100,
    target_model_update=0.01,
    policy=policy
)
dqn.compile(optimizer=Adam(lr=0.001), metrics=["mae"])
dqn.fit(env=env, nb_steps=5000, visualize=False, verbose=2)


/usr/local/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
/usr/local/lib/python3.9/site-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Training for 5000 steps ...
  456/5000: episode: 1, duration: 2.026s, episode steps: 456, steps per second: 225, episode reward: 1476.000, mean reward:  3.237 [ 0.000, 152.000], mean action: 1.596 [0.000, 3.000],  loss: 73.984195, mae: 32.208742, mean_q: 61.612010
  813/5000: episode: 2, duration: 1.669s, episode steps: 357, steps per second: 214, episode reward: 1828.000, mean reward:  5.120 [ 0.000, 152.000], mean action: 1.429 [0.000, 3.000],  loss: 149.498032, mae: 122.091827, mean_q: 176.068237
 1034/5000: episode: 3, duration: 1.016s, episode steps: 221, steps per second: 217, episode reward: 612.000, mean reward:  2.769 [ 0.000, 72.000], mean action: 1.534 [0.000, 3.000],  loss: 155.434280, mae: 88.493912, mean_q: 142.728912
 1358/5000: episode: 4, duration: 1.544s, episode steps: 324, steps per second: 210, episode reward: 1640.000, mean reward:  5.062 [ 0.000, 132.000], mean action: 1.694 [0.000, 3.000],  loss: 129.850006, mae: 145.126968, mean_q: 225.303650
 1527/5000: episod

# Simulate Game

In [ ]:
# Play the game with predicts

# Manual Game

In [ ]:
moves = ['up', 'left', 'right', 'down']
directions = ["w", "a", "d", "s"]

games = [Game2048() for _ in range(1)]
for i, game in enumerate(games):
    print(f"Game {i + 1} initial state:")
    game.render()
    while (not game.game_over):
        try: 
            currentMove = moves[directions.index(input("Next move (w, a, s , d): "))]
        except ValueError:
            print("Invalid move, try again.")
            continue

        game.move(currentMove)
        print(f"Game {i + 1} after move '{currentMove}':")
        game.render()

    print(f"Game {i + 1} is over.")


